In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from critdd import Diagram
import os

In [2]:
from reproduce_plots import collate_variants

In [3]:
full_df = collate_variants(".")

In [4]:
# 3 individuals, 2 values, 14 datasets, 30 runs
full_df.__len__() / (3  * 14 * 30)

10.0

In [5]:
full_df.loc[("ALL", "airfoil", "best_fitness")]["fitness"].mean()

np.float64(4.980373764038086)

In [6]:
full_means = full_df.groupby(level=["variant", "dataset", "individual"]).mean()

In [7]:
full_medians = full_df.groupby(level=["variant", "dataset", "individual"]).median()

In [ ]:
# want to combine
# - variant_individual
# df_temp = df.reset_index()
# df_temp["AC"] = df_temp["A"].astype(str) + "_" + df_temp["C"].astype(str)
# df_pivoted = df_temp.pivot(index="B", columns="AC", values=df.columns.tolist())

# df_cdd = full_medians.reorder_levels(["variant", "individual", "dataset"]).reset_index(["variant", "individual"])
# df_cdd["variant_individual"] = df_cdd["variant"].astype(str) + " " + df_cdd["individual"].astype(str)
# df_cdd = df_cdd.set_index("variant_individual", append=True).reorder_levels(["variant_individual", "dataset"])
# df_cdd = df_cdd.drop(columns=["variant", "individual"])

In [77]:
df_cdd_optimal = full_medians.xs("optimal_compromise", level=2).reset_index()
df_cdd_optimal = df_cdd_optimal.pivot(index="dataset", columns="variant")

In [79]:
diagram = Diagram(
    df_cdd_optimal["fitness"].to_numpy(),
    treatment_names=df_cdd_optimal["fitness"].columns,
    maximize_outcome=False
)

/home/lachlanjs/Repositories/species-ss-slim-gsgp/.venv/lib/python3.10/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)


In [91]:
diagram.average_ranks # the average rank of each treatment
# diagram.get_groups(alpha=.05, adjustment="holm")

array([3.53571429, 3.39285714, 6.42857143, 4.5       , 2.        ,
       8.39285714, 8.46428571, 4.        , 4.35714286, 9.92857143])

In [86]:
os.environ["PATH"] += ":/usr/local/texlive/2025/bin/x86_64-linux"

In [92]:
diagram.to_file(
    "example.tex",
    alpha = .05,
    adjustment = "holm",
    reverse_x = True,
    # axis_options = {"title": "critdd"},
)

In [78]:
df_cdd_optimal

fitness                                      \
variant                        ALL    ALL - AS    ALL - LS   ALL - OMS   
dataset                                                                  
airfoil                   5.034732    5.034732   16.140301    5.617082   
bike_sharing            456.539825  452.932327  759.094238  584.546570   
bioavailability          31.333086   31.226497   31.880731   31.069393   
boston                    5.067148    5.067148    6.468637    5.606843   
breast_cancer             0.276309    0.276309    0.313415    0.272946   
concrete_slump            8.107699    7.949122    8.346622    7.916509   
concrete_strength        10.755728   10.599205   13.904643   12.285376   
diabetes                 57.664381   57.678032   78.937202   59.050306   
efficiency_cooling        3.469434    3.479699    4.584055    3.613909   
efficiency_heating        3.328781    3.390787    4.958387    3.738341   
forest_fires              1.390208    1.390208    1.386615    1.385379   
parkinson_updrs           9.993967    9.996900   10.428106   10.079953   
ppb                      31.667576   31.323821   30.799860   30.077131   
resid_build_sale_price   63.681063   63.681063   79.802940   81.805809   

                                                                          \
variant                   ALL - PT         BASE    BASE + AS   BASE + LS   
dataset                                                                    
airfoil                   4.996016    57.534157    57.534157    5.563802   
bike_sharing            380.704086  1036.190369  1036.190369  584.170776   
bioavailability          30.607121    47.398724    47.398724   31.045546   
boston                    4.862624     9.426637     9.434031    5.398883   
breast_cancer             0.261209     0.355359     0.355359    0.257805   
concrete_slump            8.154112     9.716620     9.716620    7.988271   
concrete_strength         9.255052    23.576157    23.576157    9.575054   
diabetes                 57.796165   154.711189   154.711189   58.164179   
efficiency_cooling        3.329237     5.984716     5.984716    3.541753   
efficiency_heating        2.995885     6.151595     6.151595    3.215698   
forest_fires              1.379830     1.395829     1.395829    1.413241   
parkinson_updrs           9.913610    11.806863    11.806863   10.039729   
ppb                      30.402923    36.979488    36.979488   30.205249   
resid_build_sale_price   65.062351   106.894600   106.894600   80.983387   

                                                 size                    \
variant                 BASE + OMS    BASE + PT   ALL ALL - AS ALL - LS   
dataset                                                                   
airfoil                   5.268381    60.566769  20.0     21.0     11.0   
bike_sharing            618.451447  1039.016052  19.0     19.0     21.0   
bioavailability          30.605408    48.679224  20.0     19.0      9.0   
boston                    5.788058    10.092938  17.0     17.0     14.0   
breast_cancer             0.277786     0.383483  14.0     15.0     11.0   
concrete_slump            7.698610    10.256521  21.0     17.0     11.0   
concrete_strength         9.644848    25.700822  19.0     21.0     11.0   
diabetes                 71.010681   157.260887  16.0     17.0     11.0   
efficiency_cooling        3.691547     7.627475  19.0     18.0     11.0   
efficiency_heating        3.544000     7.042978  23.0     23.0     14.0   
forest_fires              1.393250     1.397230  19.0     19.0     20.0   
parkinson_updrs          10.172893    12.144903  17.0     17.0     11.0   
ppb                      29.944324    37.729963  19.0     19.0     15.0   
resid_build_sale_price   69.200588   108.291096  13.0     12.0     11.0   

                                                                     \
variant                ALL - OMS ALL - PT  BASE BASE + AS BASE + LS   
dataset                                                  

In [68]:
df_cdd = full_medians.reset_index()
df_cdd["variant_individual"] = df_cdd["variant"].astype(str) + " " + df_cdd["individual"].astype(str)
df_cdd = df_cdd.pivot(index="dataset", columns="variant_individual")
df_cdd = df_cdd.drop(columns=[("fitness", "BASE optimal_compromise"), ("size", "BASE optimal_compromise"), ("fitness", "BASE best_size"), ("size", "BASE best_size")])

In [69]:
len(df_cdd["fitness"].columns)

28

In [71]:
df_cdd["fitness"].to_numpy().shape

(14, 28)

In [73]:
df_cdd["fitness"]

variant_individual,ALL - AS best_fitness,ALL - AS best_size,ALL - AS optimal_compromise,ALL - LS best_fitness,ALL - LS best_size,ALL - LS optimal_compromise,ALL - OMS best_fitness,ALL - OMS best_size,ALL - OMS optimal_compromise,ALL - PT best_fitness,...,BASE + LS best_fitness,BASE + LS best_size,BASE + LS optimal_compromise,BASE + OMS best_fitness,BASE + OMS best_size,BASE + OMS optimal_compromise,BASE + PT best_fitness,BASE + PT best_size,BASE + PT optimal_compromise,BASE best_fitness
dataset,,,,,,,,,,,,,,,,,,,,,
airfoil,4.869670,5.709083,5.034732,5.391366,66.141197,16.140301,5.591412,5.709083,5.617082,4.935412,...,5.547357,5.622964,5.563802,5.025162,6.687323,5.268381,58.150637,64.759480,60.566769,57.007883
bike_sharing,385.041199,585.680359,452.932327,603.316345,1041.968994,759.094238,584.292236,585.307281,584.546570,375.865356,...,583.864838,584.569031,584.170776,592.567627,664.972198,618.451447,1036.008972,1041.477844,1039.016052,1035.187012
bioavailability,31.372347,30.743971,31.226497,30.050550,54.795244,31.880731,31.180829,30.974394,31.069393,31.380723,...,31.216712,30.890759,31.045546,31.538295,31.736589,30.605408,47.420227,50.349777,48.679224,46.812313
boston,4.795592,5.839152,5.067148,5.600197,10.794069,6.468637,5.505529,5.839152,5.606843,4.806698,...,5.325985,5.658345,5.398883,5.184262,7.590272,5.788058,9.665803,10.794069,10.092938,9.273700
breast_cancer,0.265083,0.307379,0.276309,0.265833,0.548263,0.313415,0.258877,0.308304,0.272946,0.259362,...,0.256427,0.278014,0.257805,0.268529,0.317215,0.277786,0.342933,0.544341,0.383483,0.314149
concrete_slump,7.696342,7.948912,7.949122,7.566355,11.701253,8.346622,7.968863,7.948911,7.916509,8.280778,...,8.201799,8.060078,7.988271,7.774341,8.781981,7.698610,9.573983,11.701253,10.256521,9.563157
concrete_strength,9.642879,13.625695,10.599205,9.415336,29.057808,13.904643,10.865689,13.546069,12.285376,8.941060,...,8.994864,11.616042,9.575054,8.656533,12.334672,9.644848,23.562177,28.381792,25.700822,22.812025
diabetes,57.396790,61.385460,57.678032,71.426083,158.898811,78.937202,57.986435,61.385460,59.050306,56.837650,...,58.191425,58.932734,58.164179,69.613491,79.115810,71.010681,154.935081,159.638855,157.260887,154.046051
efficiency_cooling,3.300719,3.841045,3.479699,3.657645,10.377571,4.584055,3.484635,3.877770,3.613909,3.287148,...,3.458689,3.580732,3.541753,3.525055,4.288557,3.691547,6.590137,10.227380,7.627475,5.749604


In [72]:
diagram = Diagram(
    df_cdd["fitness"].to_numpy(),
    treatment_names=df_cdd["fitness"].columns,
    maximize_outcome=False
)

/home/lachlanjs/Repositories/species-ss-slim-gsgp/.venv/lib/python3.10/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)


ValueError: zero_method 'wilcox' and 'pratt' do not work if x - y is zero for all elements.

In [57]:
df_cdd["fitness"]

variant_individual,ALL - AS best_fitness,ALL - AS best_size,ALL - AS optimal_compromise,ALL - LS best_fitness,ALL - LS best_size,ALL - LS optimal_compromise,ALL - OMS best_fitness,ALL - OMS best_size,ALL - OMS optimal_compromise,ALL - PT best_fitness,...,BASE + LS optimal_compromise,BASE + OMS best_fitness,BASE + OMS best_size,BASE + OMS optimal_compromise,BASE + PT best_fitness,BASE + PT best_size,BASE + PT optimal_compromise,BASE best_fitness,BASE best_size,BASE optimal_compromise
dataset,,,,,,,,,,,,,,,,,,,,,
airfoil,4.869670,5.709083,5.034732,5.391366,66.141197,16.140301,5.591412,5.709083,5.617082,4.935412,...,5.563802,5.025162,6.687323,5.268381,58.150637,64.759480,60.566769,57.007883,59.175613,57.534157
bike_sharing,385.041199,585.680359,452.932327,603.316345,1041.968994,759.094238,584.292236,585.307281,584.546570,375.865356,...,584.170776,592.567627,664.972198,618.451447,1036.008972,1041.477844,1039.016052,1035.187012,1037.273926,1036.190369
bioavailability,31.372347,30.743971,31.226497,30.050550,54.795244,31.880731,31.180829,30.974394,31.069393,31.380723,...,31.045546,31.538295,31.736589,30.605408,47.420227,50.349777,48.679224,46.812313,48.263597,47.398724
boston,4.795592,5.839152,5.067148,5.600197,10.794069,6.468637,5.505529,5.839152,5.606843,4.806698,...,5.398883,5.184262,7.590272,5.788058,9.665803,10.794069,10.092938,9.273700,9.697941,9.426637
breast_cancer,0.265083,0.307379,0.276309,0.265833,0.548263,0.313415,0.258877,0.308304,0.272946,0.259362,...,0.257805,0.268529,0.317215,0.277786,0.342933,0.544341,0.383483,0.314149,0.470136,0.355359
concrete_slump,7.696342,7.948912,7.949122,7.566355,11.701253,8.346622,7.968863,7.948911,7.916509,8.280778,...,7.988271,7.774341,8.781981,7.698610,9.573983,11.701253,10.256521,9.563157,9.971437,9.716620
concrete_strength,9.642879,13.625695,10.599205,9.415336,29.057808,13.904643,10.865689,13.546069,12.285376,8.941060,...,9.575054,8.656533,12.334672,9.644848,23.562177,28.381792,25.700822,22.812025,24.971380,23.576157
diabetes,57.396790,61.385460,57.678032,71.426083,158.898811,78.937202,57.986435,61.385460,59.050306,56.837650,...,58.164179,69.613491,79.115810,71.010681,154.935081,159.638855,157.260887,154.046051,155.874649,154.711189
efficiency_cooling,3.300719,3.841045,3.479699,3.657645,10.377571,4.584055,3.484635,3.877770,3.613909,3.287148,...,3.541753,3.525055,4.288557,3.691547,6.590137,10.227380,7.627475,5.749604,6.500166,5.984716


In [42]:
full_means.xs("best_fitness", level=2)

fitness       size
variant   dataset                                      
ALL       airfoil                   4.980374  54.666667
          bike_sharing            297.859170  41.133333
          bioavailability          31.366466  52.733333
          boston                    4.852733  43.466667
          breast_cancer             0.263210  38.200000
...                                      ...        ...
BASE + PT efficiency_heating        7.432190  55.666667
          forest_fires              1.423510  36.466667
          parkinson_updrs          11.705413  50.400000
          ppb                      37.436683  43.466667
          resid_build_sale_price  118.838105  40.933333

[140 rows x 2 columns]

In [9]:
full_means.xs("best_fitness", level=2)["fitness"].xs("BASE", level=0)

dataset
airfoil                    50.945184
bike_sharing              781.260078
bioavailability            47.645921
boston                      9.723581
breast_cancer               0.315840
concrete_slump              9.741921
concrete_strength          23.034561
diabetes                  154.483174
efficiency_cooling          7.360251
efficiency_heating          7.227877
forest_fires                1.430469
parkinson_updrs            11.567551
ppb                        37.209774
resid_build_sale_price    118.668566
Name: fitness, dtype: float64

In [10]:
base_data = full_means.xs("best_fitness", level=2)["fitness"].xs("BASE", level=0)

In [11]:
data = full_means.xs("best_fitness", level=2)["fitness"]

In [12]:
data.loc["BASE"]

dataset
airfoil                    50.945184
bike_sharing              781.260078
bioavailability            47.645921
boston                      9.723581
breast_cancer               0.315840
concrete_slump              9.741921
concrete_strength          23.034561
diabetes                  154.483174
efficiency_cooling          7.360251
efficiency_heating          7.227877
forest_fires                1.430469
parkinson_updrs            11.567551
ppb                        37.209774
resid_build_sale_price    118.668566
Name: fitness, dtype: float64

In [13]:
full_means.xs("best_fitness", level=2)["fitness"].loc["BASE"]

dataset
airfoil                    50.945184
bike_sharing              781.260078
bioavailability            47.645921
boston                      9.723581
breast_cancer               0.315840
concrete_slump              9.741921
concrete_strength          23.034561
diabetes                  154.483174
efficiency_cooling          7.360251
efficiency_heating          7.227877
forest_fires                1.430469
parkinson_updrs            11.567551
ppb                        37.209774
resid_build_sale_price    118.668566
Name: fitness, dtype: float64

In [14]:
all_data = full_means.xs("best_fitness", level=2)["fitness"].xs("ALL", level=0)

In [15]:
all_data - base_data

dataset
airfoil                   -45.964810
bike_sharing             -483.400908
bioavailability           -16.279454
boston                     -4.870849
breast_cancer              -0.052630
concrete_slump              0.213240
concrete_strength         -13.383774
diabetes                  -96.856588
efficiency_cooling         -4.054467
efficiency_heating         -4.154027
forest_fires               -0.008386
parkinson_updrs            -1.646736
ppb                         4.479856
resid_build_sale_price    -56.771727
Name: fitness, dtype: float64

In [16]:
plt.plot(np.arange(len(all_data)), all_data - base_data)

Error in callback <function _draw_all_if_interactive at 0x7f0621f52f80> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: 'pdflatex' not found; install it or change rcParams['pgf.texsystem'] to an available TeX implementation